In [34]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [6]:
folder_path = '/home/melissa.lopez/GlitchBank/LargeAnalysis/O3aGB/FinalData/'
detector = 'L1'

In [117]:
# Define some lists
blip_files, fast_scattering_files, injection_files = list(), list(), list()
koyfish_files, lowfreq_files, tomte_files = list(), list(), list()
whistle_files = list()

# Iterate over all files in the given folder
for filename in np.sort(os.listdir(folder_path)):
    # Check if the file is a CSV file
    if filename.endswith(".csv"):
        #Check what type of data the file contains and put in in the corresponding list.
        if filename.startswith("Blip_" + detector):
            blip_files.append((os.path.join(folder_path, filename)))                     
        elif filename.startswith("Injections_" + detector):
            injection_files.append((os.path.join(folder_path, filename)))       
        elif filename.startswith("Koi_Fish_" + detector):
            koyfish_files.append((os.path.join(folder_path, filename)))
        elif filename.startswith("Low_Frequency_Burst_" + detector):
            lowfreq_files.append((os.path.join(folder_path, filename)))
        elif filename.startswith("Tomte_" + detector):
            tomte_files.append((os.path.join(folder_path, filename)))
        elif filename.startswith("Whistle_" + detector):
            whistle_files.append((os.path.join(folder_path, filename))) 
        elif filename.startswith("Fast_Scattering_" + detector) and (detector != "V1"):
            fast_scattering_files.append((os.path.join(folder_path, filename)))

In [12]:
path = injection_files[0]
df = pd.read_csv(path)

In [113]:
df1_grouped = df.groupby(['Cluster ID', 'Cluster time'])

In [114]:
def averageFeature(cols, group_name, tmp):
    feature = np.zeros((1, len(cols)))

    for c, col in enumerate(cols):
        feature[0, c] = (tmp[col] * weights).sum() / weights.sum()
    feature = pd.DataFrame(feature, columns=cols)

    return feature

In [116]:
drop_cols = ['Unnamed: 0', 'ifo', 'Trigger ID', 'Trigger time', 'Cluster ID', 'Cluster time', 'Template ID']
k = 0
cluster_id, cluster_time = list(), list()
for group_name, df_group in df1_grouped:
    print(group_name)
    cluster_id.append(group_name[0])
    cluster_time.append(group_name[1])
    tmp = df_group.sort_values(by='Trigger time')
    time = tmp['Trigger time']
    tmp = tmp.drop(drop_cols, axis=1)
    weights = tmp['SNR'] / np.max(tmp['SNR'])
    print(len(tmp['SNR']), np.mean(tmp['SNR']))
    cols = tmp.columns
    feature = averageFeature(cols, group_name, tmp)
    
    if k == 0:
        features = feature
    else:
        features = pd.concat([features, feature])
    k = k + 1
    display(features)
    if k == 2:
        break

IDs = pd.DataFrame({'Cluster ID':cluster_id, 'Cluster time':cluster_time})

(2, 1238166114.4367027)
64 7.5837232859374994


,SNR,Chisq,Mass_1,Mass_2,Spin1z,Spin2z
0,7.850731,2.769055,174.809988,37.57883,-0.28955,0.06697


(2, 1244451019.3194437)
117 16.27856853931624


,SNR,Chisq,Mass_1,Mass_2,Spin1z,Spin2z
0,7.850731,2.769055,174.809988,37.578830,-0.289550,0.066970
0,16.747768,3.476546,62.979454,11.773144,-0.483233,-0.055422


In [107]:
df_group['Cluster ID'].unique()

array([2])

In [80]:
cols_ = ['group']
cols_.extend(cols)
cols_

['group', 'SNR', 'Chisq', 'Mass_1', 'Mass_2', 'Spin1z', 'Spin2z']

In [118]:
pd.read_csv('dummy_IDs.csv')

,Unnamed: 0,Cluster ID,Cluster time
0,0,2,1.238166e+09
1,1,2,1.244451e+09
2,2,3,1.238166e+09
3,3,3,1.250674e+09
4,4,5,1.240334e+09
...,...,...,...
159234,159234,27214,1.247737e+09
159235,159235,27215,1.247737e+09
159236,159236,27218,1.247737e+09
159237,159237,27219,1.247737e+09


In [139]:
dt = pd.read_csv('/home/melissa.lopez/GlitchBank/LargeAnalysis/O3aGB/AvDataFrames/Av_Injections_L1_O3a.csv')

In [140]:
dt

,SNR,Chisq,Mass_1,Mass_2,Spin1z,Spin2z
0,7.850731,2.769055,174.809988,37.578830,-0.289550,0.066970
1,16.747768,3.476546,62.979454,11.773144,-0.483233,-0.055422
2,4.469691,0.928928,37.415399,16.995580,0.758052,-0.100917
3,10.452025,3.689212,99.204812,16.232065,-0.458298,0.375759
4,7.595686,0.540775,422.670937,110.160999,0.328761,0.208082
...,...,...,...,...,...,...
159234,9.912576,2.755996,70.259355,10.760917,-0.338562,0.104626
159235,4.344459,0.908955,45.405682,4.775333,0.953063,-0.934331
159236,4.058321,1.080616,45.997225,6.359614,0.955899,0.190847
159237,4.301000,0.834829,44.990089,5.676049,0.954672,0.078123


In [141]:
!ls /home/melissa.lopez/GlitchBank/LargeAnalysis/O3aGB/AvDataFrames/

Av_Blip_L1_O3a.csv		  Av_Koi_Fish_L1_O3b.csv
Av_Blip_L1_O3a_ID.csv		  Av_Koi_Fish_L1_O3b_ID.csv
Av_Blip_L1_O3b.csv		  Av_Low_Frequency_Burst_L1_O3a.csv
Av_Blip_L1_O3b_ID.csv		  Av_Low_Frequency_Burst_L1_O3a_ID.csv
Av_Fast_Scattering_L1_O3a.csv	  Av_Low_Frequency_Burst_L1_O3b.csv
Av_Fast_Scattering_L1_O3a_ID.csv  Av_Low_Frequency_Burst_L1_O3b_ID.csv
Av_Fast_Scattering_L1_O3b.csv	  Av_Tomte_L1_O3a.csv
Av_Fast_Scattering_L1_O3b_ID.csv  Av_Tomte_L1_O3a_ID.csv
Av_Injections_L1_O3a.csv	  Av_Tomte_L1_O3b.csv
Av_Injections_L1_O3a_ID.csv	  Av_Tomte_L1_O3b_ID.csv
Av_Injections_L1_O3b.csv	  Av_Whistle_L1_O3a.csv
Av_Injections_L1_O3b_ID.csv	  Av_Whistle_L1_O3a_ID.csv
Av_Koi_Fish_L1_O3a.csv		  Av_Whistle_L1_O3b.csv
Av_Koi_Fish_L1_O3a_ID.csv	  Av_Whistle_L1_O3b_ID.csv
